In [15]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from azureml.core.model import Model

In [16]:
ws = Workspace.from_config()

experiment_name = 'automl-crm-classification'

experiment=Experiment(ws, experiment_name)

In [17]:
last_run = AutoMLRun(experiment,"AutoML_319daac5-520d-43d7-8dbb-1bea312c63fa")

## Register Model

Register Model
You can add tags and descriptions to your models. Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.

You can optionally register a specific iteration of an experiment, or select the best run based off a different primary_metric.

In [18]:
model = last_run.register_model(
    description="best model from CRM automl demo",
    tags={"project": "demo", "author": "Vish"}
#     iteration=8,
#     metric="AUC_weighted"
)

In [19]:
model = Model(ws, 'AutoML319daac558')

## Deploy Model via Docker/Flask

Specify environment requirements and scoring file.  Both of these are generated automatically from Azure.

In [7]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('crm-environment')
environment.python.conda_dependencies = CondaDependencies("conda_env_v_1_0_0.yml")

In [8]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="scoring_file_v_1_0_0.py",
                                   environment=environment)

### Create and deploy the local Docker image


In [9]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
2020/03/19 18:00:13 Downloading source code...
2020/03/19 18:00:15 Finished downloading source code
2020/03/19 18:00:15 Creating Docker network: acb_default_network, driver: 'bridge'
2020/03/19 18:00:16 Successfully set up Docker network: acb_default_network
2020/03/19 18:00:16 Setting up Docker configuration...
2020/03/19 18:00:17 Successfully set up Docker configuration
2020/03/19 18:00:17 Logging in to registry: dswdsectionaeb42fd2d.azurecr.io
2020/03/19 18:00:18 Successfully logged into dswdsectionaeb42fd2d.azurecr.io
2020/03/19 18:00:18 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/03/19 18:00:18 Scanning for dependencies...
2020/03/19 18:00:19 Successfully scanned dependencies
2020/03/19 18:00:19 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB
Step 1/15 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a769

  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=4698116ae43906b4efa26a06c5155780f22b8155f4aa9be032e1abd93e827217
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66714 sha256=8eac1f2fdd9d53da2ac4cf2bbf2a151e90915d9ebb92ae2ca990468635c8e1a6
  Stored in directory: /root/.cache/pip/wheels/94/0f/ec/66085641573800014bb0c8b657f3366eff641c42df79abbfe9
  Created wheel for shap: filename=shap-0.29.3-cp36-cp36m-linux_x86_64.whl size=349369 sha256=e47666f9afc5ee6e69c1b069b583ea554541d555d5031ee52375c37fde563128
  Stored in directory: /root/.cache/pip/wheels/1c/22/4b/ae66ee74140d6abc86f4eda1917915865a736bf9c138338c5f
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=df362d98281faf57a9b962a53284c9a34d5223cd70c1dbfa2fa30d8fd1288c6f
  Stored in directory: /root/.cache/pip/wheels/

  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-py3-none-any.whl size=77592 sha256=a6afc0edeb84529736aa1a8e09986bb229be8bdc67c20f61c21baccf802abb61
  Stored in directory: /root/.cache/pip/wheels/35/8d/af/a922cb18800b31fadac3523cadf6c1efdf233b788fe7a4da70
Successfully built dill wrapt shap json-logging-py fusepy py-cpuinfo smart-open JsonSir JsonForm liac-arff PyYAML fire backcall pyrsistent termcolor googleapis-common-protos
ERROR: fbprophet 0.5 requires setuptools-git>=1.2, which is not installed.
ERROR: xarray 0.15.0 has requirement pandas>=0.25, but you'll have pandas 0.23.4 which is incompatible.
ERROR: azureml-automl-runtime 1.0.83 has requirement azureml-automl-core==1.0.83, but you'll have azureml-automl-core 1.0.83.1 which is incompatible.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3
  Attempting uninstall: scipy
    Found existing in

807c07f9a5e8: Pushed
e1171d4d60ca: Pushed
6ef1a8ae63b7: Pushed

340dc32eb998: Pushed
85389f9ead9e: Pushed
ccdb13a20bf2: Pushed
9513cdf4e497: Pushed
0e259b09e5f4: Pushed
7f083f9454c0: Pushed
f2608f66a0e3: Pushed
df18b66efaa6: Pushed
29f36b5893dc: Pushed

68dc08739d11: Pushed
latest: digest: sha256:4a7186ee04f7d89892b76f4dbf73297c4e0e4134e8a68457028aaf7ded71f205 size: 3883
2020/03/19 18:11:30 Successfully pushed image: dswdsectionaeb42fd2d.azurecr.io/azureml/azureml_bf18c0c768fad1eb213dcdf910073577:latest
2020/03/19 18:11:30 Step ID: acb_step_0 marked as successful (elapsed time in seconds: 470.482755)
2020/03/19 18:11:30 Populating digests for step ID: acb_step_0...
2020/03/19 18:11:32 Successfully populated digests for step ID: acb_step_0
2020/03/19 18:11:32 Step ID: acb_step_1 marked as successful (elapsed time in seconds: 201.869780)
2020/03/19 18:11:32 The following dependencies were found:
2020/03/19 18:11:32 
- image:
    registry: dswdsectionaeb42fd2d.azurecr.io
    repository: a

In [ ]:
input_sample = pd.DataFrame(data=[{'a_loc_Central': 0, 'a_loc_East': 1, 'acc_size': 2, 
                                   'opp_size': 913641, 'p1_devs': 108, 
                                   'p1p2_2': 6.0, 'p2_devs': 18, 
                                   'p_loc_Central': 1, 'p_loc_East': 0, 
                                   'partner_driven': 1, 'partner_size': 2}])


Run the inference model via the azureml sdk

In [32]:
import json



sample_input = json.dumps({
    'data': [
        [ 0, 1, 2,
          1000000, 50,
          2.0, 25,
          1, 0,
          1, 2],
        [ 1, 0, 2,
          10000, 300,
          3.0, 2,
          1, 0,
          0, 1]
    ]
})

sample_input = bytes(sample_input, encoding='utf-8')

local_service.run(input_data=sample_input)

Making a scoring call...
Scoring result:
{"result": [1, 0]}


'{"result": [1, 0]}'